In [ ]:
import h5py
import boto3
import pandas as pd
import os
import numpy as np

import urllib
import ftplib
import sys
import h5py
import numpy as np
from osgeo import gdal
from osgeo import gdal_array
from osgeo import osr

import requests as req

In [ ]:
s3 = boto3.client('s3')

In [ ]:
avail_files = ["SMAP_L4_SM_gph_20170806T013000_Vv3030_001.h5"]
bucket = "resourcewatch/raster/foo_001_soil_moisture_active_passive/"
for fileObj in avail_files:
    s3.download_file("wri-public-data", bucket + fileObj, fileObj)

In [ ]:
# Command to get
# wget --load-cookies ./.urs_cookies --save-cookies ./.urs_cookies --keep-session-cookies --no-check-certificate --auth-no-challenge=on -r --reject "index.html*" -np -e robots=off https://n5eil01u.ecs.nsidc.org/SMAP/SPL3SMA.003/2015.04.13/SMAP_L3_SM_A_20150413_R13080_002.h5 -P SMAP_L3_SM_A_20150413_R13080_002.h5  

# ^ doesn't work... downloading directly does

os.listdir()
f = h5py.File("SMAP_L2_SM_AP_01056_D_20150413T184423_R13080_001.h5", "r")
# Unable to open file - corrupted

In [ ]:
!gdalinfo SMAP_L4_SM_gph_20170806T013000_Vv3030_001.h5

In [ ]:
# List data groups
print(list(f.items()))

dataset = f["Soil_Moisture_Retrieval_Data_3km"]

# No attributes
print(list(dataset.attrs))
# List keys
print(list(dataset.keys()))

# Retrieve dataset
data = dataset.get("soil_moisture_v_3km")
lats = dataset.get("latitude_3km")
lons = dataset.get("longitude_3km")

# See attributes of dataset
print(list(data.attrs.keys()), "\n")
for key in data.attrs.keys():
    print(key, ": ",  data.attrs[key])


    
# Inspect data 
print('\n')
print("data shape: ", data.shape)

# Select only valid vals
#valid_vals = np.where(data[:]==-9999., False, data[:])
invalid_data_ind = np.where(data[:]==-9999.)

valid_data_ind = np.where(data[:]==-9999.)

print('\n')
valid_data = np.delete(data, invalid_data_ind)
valid_lats = np.delete(lats, invalid_data_ind)
valid_lons = np.delete(lons, invalid_data_ind)
print(len(valid_data))

In [ ]:
def HDF5toGeotiff(data, lat, lon, fileLocation, fileName):
    parameter = nc_vars[3]
    lats = nc_fid.variables['latitude'][:]  # extract/copy the data
    lons = nc_fid.variables['longitude'][:]
    times = nc_fid.variables['time'][:]
    timeUnit = nc_fid.variables["time"].getncattr("units")
    timeNormal =[]
    for time in times:
        if timeUnit == ("days since 1900-01-01 00:00:00") or (timeUnit =="Days since 1900-01-01"):
            timeNormal.append(datetime.datetime(1900,1,1) + datetime.timedelta(days=time))
        elif timeUnit == "days since 1901-01-01 00:00:00":
            timeNormal.append(datetime.datetime(1901,1,1) + datetime.timedelta(days=time))
        else:
            print "Error"
            timeNormal.append(-9999)
            
    for i in range(0,len(timeNormal)):
        #print timeNormal[i].year
        Z = nc_fid.variables[parameter][i, :, :]
        Z[Z<-9990]= -9999
        Z[Z>1e19] = -9999
        outputFilename = netCDFInputBaseName + "I%0.3dY%0.2dM%0.2d.tif" %(i,timeNormal[i].year,timeNormal[i].month)
        writefilename = os.path.join(EC2_OUTPUTPATH,outputFilename)
        writeFile(writefilename,geotransform,geoproj,Z)
    

In [ ]:
def writeFile(filename,geotransform,geoprojection,data):
    (x,y) = data.shape
    format = "GTiff"
    driver = gdal.GetDriverByName(format)
    # you can change the dataformat but be sure to be able to store negative values including -9999
    dst_datatype = gdal.GDT_Float32
    dst_ds = driver.Create(filename,y,x,1,dst_datatype, [ 'COMPRESS=LZW' ])
    dst_ds.GetRasterBand(1).SetNoDataValue(-9999)
    dst_ds.GetRasterBand(1).WriteArray(data)
    dst_ds.SetGeoTransform(geotransform)
    dst_ds.SetProjection(geoprojection)
    dst_ds = None
    return 1

In [ ]:
!gdal_translate -of GTiff -a_srs '+proj=longlat +datum=WGS84 +no_defs ' 'HDF5:"SMAP_L2_SM_AP_01056_D_20150413T184423_R13080_001.h5"://Soil_Moisture_Retrieval_Data_3km' SMAP.tif


In [ ]:
from osgeo import gdal
import numpy as np

def hdf_subdataset_extraction(hdf_file, dst_dir, subdataset):
    """unpack a single subdataset from a HDF5 container and write to GeoTiff"""
    # open the dataset
    hdf_ds = gdal.Open(hdf_file, gdal.GA_ReadOnly)
    band_ds = gdal.Open(hdf_ds.GetSubDatasets()[subdataset][0], gdal.GA_ReadOnly)

    # read into numpy array
    band_array = band_ds.ReadAsArray().astype(np.int16)

    # convert no_data values
    # band_array[band_array == -28672] = -32768

    # build output path
    band_path = os.path.join(dst_dir, os.path.basename(os.path.splitext(hdf_file)[0]) + "-sd" + str(subdataset+1) + ".tif")

    # write raster
    out_ds = gdal.GetDriverByName('GTiff').Create(band_path,
                                                  band_ds.RasterXSize,
                                                  band_ds.RasterYSize,
                                                  1,  #Number of bands
                                                  gdal.GDT_Int16,
                                                  ['COMPRESS=LZW', 'TILED=YES'])
    out_ds.SetGeoTransform(band_ds.GetGeoTransform())
    out_ds.SetProjection(band_ds.GetProjection())
    out_ds.GetRasterBand(1).WriteArray(band_array)
    out_ds.GetRasterBand(1).SetNoDataValue(-32768)

    out_ds = None  #close dataset to write to disc

In [ ]:
#hdf_ds = gdal.Open('HDF5:"SMAP_L2_SM_AP_01056_D_20150413T184423_R13080_001.h5"://Soil_Moisture_Retrieval_Data_3km', gdal.GA_ReadOnly)
hdf_ds = gdal.Open("SMAP_L2_SM_AP_01056_D_20150413T184423_R13080_001.h5", gdal.GA_ReadOnly)
print(hdf_ds.GetSubDatasets())
hdf_ds.GetMetadata()

In [ ]:
# From: https://earthdatascience.org/tutorials/create-raster-from-smap-soil-moisture-data/

which_group = "Geophysical_Data"
datasets = ['sm_rootzone_wetness']
file_path = "SMAP_L4_SM_gph_20170806T013000_Vv3030_001.h5"

def smap2raster(inputFile, group, dataset):
    """Converts SMAP data to a Raster object
Input:  
    inputFile - SMAP data file
    group - The groupt containing the dataset we want to pull data from
    dataset - Which specific dataset we want to pull data from
Output: 
    A raster image in .tif format, saved to the current working directory
    """
    #Read in the SMAP file in h5 format
    h5File = h5py.File(inputFile, 'r')
    
    #Get the data from the specific group/dataset
    data = h5File.get(group + '/' + dataset)
    lat = h5File.get('cell_lat')
    lon = h5File.get('cell_lon')
    
    #Convert this data into numpy arrays
    np_data = np.array(data)
    np_lat = np.array(lat)
    np_lon = np.array(lon)
    
    #Get the spatial extents of the data
    num_cols = float(np_data.shape[1])
    num_rows = float(np_data.shape[0])
    xmin = np_lon.min()
    xmax = np_lon.max()
    ymin = np_lat.min()
    ymax = np_lat.max()
    xres = (xmax - xmin)/num_cols
    yres = (ymax - ymin)/num_rows
    
    #Set up the transformation necessary to create the raster
    geotransform = (xmin, xres, 0, ymax, 0, -yres)
    
    #Create the raster object with the proper coordinate encoding and geographic transformation
    driver = gdal.GetDriverByName('GTiff')
    raster = driver.Create(dataset+'Raster.tif', int(num_cols), int(num_rows), 1, gdal.GDT_Float32)
    raster.SetGeoTransform(geotransform)
    srs = osr.SpatialReference()
    srs.ImportFromEPSG(4326)
    
    #Export and write the data array to the raster
    raster.SetProjection( srs.ExportToWkt() )
    raster.GetRasterBand(1).WriteArray(np_data)
    h5File.close()

#Create an array of the datasets we want to use
#Replace 'snow_mass' and 'snow_depth' with the datasets you want to use
# datasets = ['sm_surface_wetness', 'soil_temp_layer2']

#Loop through the datasets and create individual rasters from them
for i in range(0, len(datasets)):
    smap2raster(file_path, which_group, datasets[i])

In [ ]:
dataset = 'sm_rootzone_wetness'
inputFile = "SMAP_L4_SM_gph_20170806T013000_Vv3030_001.h5"
group = "Geophysical_Data"

h5File = h5py.File(inputFile, 'r')
#Get the data from the specific group/dataset
data = h5File.get(group + '/' + dataset)
np_data = np.array(data)
print(np_data.shape)

lat = h5File.get('cell_lat')
lon = h5File.get('cell_lon')
print(len(lat))
print(len(lon))

In [ ]:
file_path = 'SMAP_L4_SM_gph_20170806T013000_Vv3030_001.h5'
h5file = h5py.File(file_path, 'r') 

which_group = 'Geophysical_Data'
group = h5file.get(which_group)
datasets = np.array(group)
print(datasets)